In [0]:
%sql
CREATE CONNECTION IF NOT EXISTS earthquake_connn TYPE HTTP OPTIONS (
  host = 'https://earthquake.usgs.gov', port = 443, base_path = '/earthquakes/feed/v1.0/',
  bearer_token = 'na'
);

In [0]:
dbutils.widgets.text('catalog_name','youtude_dev','youtude_dev')
catalog_name = dbutils.widgets.get('catalog_name')

In [0]:
%py
spark.sql(f"use catalog {catalog_name}")
spark.sql(f"use schema bronze")
spark.sql("create volume if not exists earthquake_data")

In [0]:
from databricks.sdk import WorkspaceClient
import requests
import json
import datetime

w = WorkspaceClient()
conn = w.connections.get("earthquake_connn")

base_url = f"{conn.options['host'].rstrip('/')}{conn.options['base_path'].rstrip('/')}"
url = f"{base_url}/summary/all_day.geojson"

response = requests.get(url, timeout=30)
if response.status_code != 200:
    raise exception(f"Error in getting data from {url}")

print("Status:", response.status_code)
data = response.json()
print("Top keys:", data.keys())
current_date = datetime.datetime.now().strftime("%Y-%m-%d")
dbutils.fs.put(
    f"/Volumes/{catalog_name}/bronze/earthquake_data/earthquake_data_{current_date}.json",
    json.dumps(data),
    overwrite=True,
)